# Άσκηση Deep Feed Forward NN

Στην άσκηση αυτή θέλουμε να φτιάξουμε ένα πολυεπίπεδο νευρωνικό δίκτυο με 5 επίπεδα (4 κρυφά και 1 επίπεδο εξόδου) για να επιλύσουμε το MNIST. Τα κρυφά επίπεδα θα επιλέξουμε να έχουν 200, 100, 60 και 30 νευρώνες αντίστοιχα και [σιγμοειδή](https://en.wikipedia.org/wiki/Sigmoid_function) συνάρτηση ενεργοποίησης. Το επίπεδο εξόδου θα έχει συνάρτηση ενεργοποίησης την [softmax](https://en.wikipedia.org/wiki/Softmax_function). Θα χρησιμοποιήσουμε την [cross-entropy loss](https://en.wikipedia.org/wiki/Cross_entropy) ως συνάρτηση κόστους και ένα απλό [Mini-Batch Gradient Descent](https://en.wikipedia.org/wiki/Gradient_descent). Θέστε `batch_size = 128`.

In [1]:
#!pip install --upgrade tensorflow
import tensorflow as tf
import matplotlib.pyplot as plt
import time
%matplotlib inline

Κατασκευάστε το Νευρωνικό Δίκτυο που περιγράφεται παραπάνω και εκπαιδεύστε το για 400 εποχές. Έπειτα απαντήστε στα παρακάτω ερωτήματα.

### Ερωτήματα:

- Πόσες παραμέτρους έχει συνολικά το Νευρωνικό Δίκτυο;
- Γιατί το accuracy του test set είναι μικρότερο από αυτό του tranining set;

In [2]:
#έτσι ελέγχουμε ότι τρέχουμε στη GPU

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
#@title
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist

# load mnist dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# compute the number of labels
num_labels = len(np.unique(y_train))

# convert to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
  
# image dimensions (assumed square)
image_size = x_train.shape[1]
input_size = image_size * image_size

# resize and normalize
x_train = np.reshape(x_train, [-1, input_size])
x_train = x_train.astype('float32') / 255
x_test = np.reshape(x_test, [-1, input_size])
x_test = x_test.astype('float32') / 255

# network parameters
batch_size = 128

model = Sequential()
model.add(Dense(200, input_dim=input_size))
model.add(Activation('sigmoid'))

model.add(Dense(100))
model.add(Activation('sigmoid'))

model.add(Dense(60))
model.add(Activation('sigmoid'))

model.add(Dense(30))
model.add(Activation('sigmoid'))

model.add(Dense(num_labels))

# this is the output for one-hot vector
model.add(Activation('softmax'))

# Εναλλακτικός τρόπος
model = Sequential([
  Dense(200, input_dim=input_size,activation='sigmoid'),
  Dense(100, activation='sigmoid'),
  Dense(60, activation='sigmoid'),
  Dense(30, activation='sigmoid'),
  Dense(num_labels, activation='softmax')
])


model.summary()
plot_model(model, to_file='mlp-mnist.png', show_shapes=True)

# loss function for one-hot vector
# use of sgd optimizer
# accuracy is good metric for classification tasks
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])


epochs = 400

# train the network
%timeit -n 1 -r 1 model.fit(x_train, y_train, epochs=400, batch_size=batch_size)

# validate the model on test dataset to determine generalization
_, acc = model.evaluate(x_test,
                        y_test,
                        batch_size=batch_size,
                        verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))


11493376/11490434 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 200)               157000    
_________________________________________________________________
dense_6 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_7 (Dense)              (None, 60)                6060      
_________________________________________________________________
dense_8 (Dense)              (None, 30)                1830      
_________________________________________________________________
dense_9 (Dense)              (None, 10)                310       
Total params: 185,300
Trainable params: 185,300
Non-trainable params: 0
_________________________________________________________________
Epoch 1/400
469/469 [============================